In [1]:
import numpy as np
from pvlib import pvsystem, singlediode
import sys
module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
sys.path.insert(0, module_path)
from ipv_workbench.utilities import utils
import time

In [2]:
Geff = np.zeros((264,10)) + 500
Tcell = np.zeros((264,10)) + 25

In [3]:
submodule_map, diode_map, subcell_map = utils.read_map_excel("/Users/jmccarty/Desktop/bano_simulations/resources/map_files/cdte_landscape_maps.xlsx")

In [79]:

param_time = []
diode_time = []

for n in np.arange(0,100):

    submodules = np.unique(submodule_map)
    diodes = np.unique(diode_map)

    evaluated_voltages = np.linspace(0.95 * -5.5,
                                        214.3 * 1.05,
                                        1000)
    submodule_i = []
    submodule_v = []
    for submodule_key in submodules:
        _0 = time.time()
        submodule_mask = submodule_map == submodule_key
        submodule_rowmask = submodule_mask.flatten()

        submodule_diode = np.array(diode_map)[submodule_rowmask, :]
        submodule_subcell = np.array(subcell_map)[submodule_rowmask, :]

        submodule_irrad = Geff[submodule_rowmask, :]
        submodule_temp = Tcell[submodule_rowmask, :]

        diode_i = []
        diode_v = []
        _1 = time.time()
        # print("masking1", _1 - _0)
        for diode_key in diodes:
            diode_mask = submodule_diode == diode_key
            diode_rowmask = diode_mask.flatten()

            submodule_subdiode_subcell = submodule_subcell[diode_rowmask, :]

            submodule_subdiode_irrad = submodule_irrad[diode_rowmask, :]
            submodule_subdiode_temp = submodule_temp[diode_rowmask, :]

            cells_i = []
            cells_v = []
            _2 = time.time()
            # print("masking2", _2 - _1)
            for subcell_key in np.unique(submodule_subdiode_subcell):
                submodule_subdiode_subcell_mask = submodule_subdiode_subcell == subcell_key
                submodule_subdiode_subcell_irrad = submodule_subdiode_irrad[submodule_subdiode_subcell_mask]
                submodule_subdiode_subcell_temp = submodule_subdiode_temp[submodule_subdiode_subcell_mask]

                num_subcells = 10

                sde_args = np.vectorize(pvsystem.calcparams_desoto)(submodule_subdiode_subcell_irrad,
                                                                    submodule_subdiode_subcell_temp,
                                                                    alpha_sc=0.00137,
                                                                    a_ref=7.402658,
                                                                    I_L_ref=2.509123,
                                                                    I_o_ref=0.0,
                                                                    R_sh_ref=1065.831543,
                                                                    R_s=8.185414)
                _3 = time.time()
                # print("Param calc", _3 - _2)
                param_time.append(_3 - _2)
                sde_args = np.array(sde_args)
                # Use Bishop's method to calculate points on the IV curve with V ranging
                # from the reverse breakdown voltage to open circuit
                kwargs = {
                    'breakdown_factor': 0.002,
                    'breakdown_exp': 3.28,
                    'breakdown_voltage': -5.5,
                }

                i_subcell = []
                v_subcell = []


                
                for n in range(0, num_subcells):
                    i_, v_, p_ = singlediode.bishop88(evaluated_voltages,
                                                        *sde_args[:, n],
                                                        **kwargs
                                                        )
                    
                _4 = time.time()
                # print("Single Diode",_4 - _3)
                diode_time.append(_4 - _3)


KeyboardInterrupt: 

In [80]:
print(np.sum(param_time))
print(np.sum(diode_time))

22.46795392036438
44.59918808937073


In [74]:

for t in range(0,100):
    i_subcell = []
    v_subcell = []
    for n in range(0, num_subcells):
        i_, v_, p_ = singlediode.bishop88(evaluated_voltages,
                                            *sde_args[:, n],
                                            **kwargs
                                            )
        i_ = i_ / 10
        v_ = v_ / 264
        i_subcell.append(i_)
        v_subcell.append(v_)

In [65]:
vfun = np.vectorize(singlediode.bishop88)


In [73]:
for t in range(0,100):
    i_, v_, p_ = vfun([evaluated_voltages.reshape(1000,1).tolist()],
                        sde_args[0],
                        sde_args[1],
                        sde_args[2],
                        sde_args[3],
                        sde_args[4], 
                        0,
                        np.inf,
                        kwargs['breakdown_factor'],
                        kwargs['breakdown_voltage'],
                        kwargs['breakdown_exp']
                        )
    i_subcell = i_[0] / 10
    v_subcell = v_[0] / 264
    # p_ = p_[0]

In [64]:
r[0][0]

array([[1.3477479 , 1.3477479 , 1.3477479 , ..., 1.3477479 , 1.3477479 ,
        1.3477479 ],
       [1.26868257, 1.26868257, 1.26868257, ..., 1.26868257, 1.26868257,
        1.26868257],
       [1.26007308, 1.26007308, 1.26007308, ..., 1.26007308, 1.26007308,
        1.26007308],
       ...,
       [1.14921931, 1.14921931, 1.14921931, ..., 1.14921931, 1.14921931,
        1.14921931],
       [1.14911119, 1.14911119, 1.14911119, ..., 1.14911119, 1.14911119,
        1.14911119],
       [1.14900307, 1.14900307, 1.14900307, ..., 1.14900307, 1.14900307,
        1.14900307]])